使用线程允许一个程序在同一个进程空间中并发运行多个操作。

In [1]:
import threading

def worker(num):
    """thread worker function"""
    print(f'Worker {num}')
    print(threading.current_thread().getName())

threads = []
for i in range(5):
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()

Worker 0
Thread-4
Worker 1
Thread-5
Worker 2
Thread-6
Worker 3
Thread-7
Worker 4
Thread-8


In [2]:
import logging
import time
# logging 模块支持在日志信息中写入线程的名字，你可以用格式化代码 %(threadName)s 来得到它。
def worker():
    logging.debug('Starting')
    time.sleep(0.2)
    logging.debug('Exit')
    
def service():
    logging.debug('Starting')
    time.sleep(0.5)
    logging.debug('Exit')
    
t1 = threading.Thread(target=worker)
t2 = threading.Thread(target=service)
logging.basicConfig(level=logging.DEBUG, format="[%(levelname)s] (%(threadName)-10s) %(message)s")

t1.start()
t2.start()

[DEBUG] (Thread-9  ) Starting
[DEBUG] (Thread-10 ) Starting
[DEBUG] (Thread-9  ) Exit
[DEBUG] (Thread-10 ) Exit


## 守护线程

In [15]:
def daemon():
    logging.debug('Daemon Starting')
    time.sleep(0.2)
    logging.debug('Daemon Exit')
    
def non_daemon():
    logging.debug('Starting')
    logging.debug('Exit')
    
t1 = threading.Thread(target=daemon, daemon=True, name='daemon')
t2 = threading.Thread(target=non_daemon, name='non-daemon')
logging.basicConfig(level=logging.DEBUG, format="[%(levelname)s] (%(threadName)-10s) %(message)s")

t1.start()
t2.start()

[DEBUG] (daemon    ) Daemon Starting
[DEBUG] (non-daemon) Starting
[DEBUG] (non-daemon) Exit
[DEBUG] (daemon    ) Daemon Exit


```
➜  practise git:(master) ✗ /usr/local/opt/python3/bin/python3.6 /Users/hejl/local/practise/standard_library/test.py
[DEBUG] (daemon    ) Daemon Starting
[DEBUG] (non-daemon) Starting
[DEBUG] (non-daemon) Exit
```

t1.join()则会使守护线程持续执行下去，join参数timeout设置后，若超时线程并未结束，则join会返回，不会继续等待。
## 枚举所有线程

In [19]:
import random

def worker():
    pause = random.randint(1,5)/10
    logging.debug('Daemon Starting sleep %0.2f', pause)
    time.sleep(pause)
    logging.debug('Daemon Exit')
    
logging.basicConfig(level=logging.DEBUG, format="[%(levelname)s] (%(threadName)-10s) %(message)s")
for i in range(3):
    t = threading.Thread(target=worker, daemon=True)
    t.start()
    
main_thread = threading.main_thread()

print(threading.enumerate())
for t in threading.enumerate():
    if t is main_thread:
        continue
    logging.debug('%s joining' , t.getName())
    t.join()

[DEBUG] (Thread-64 ) Daemon Starting sleep 0.30
[DEBUG] (Thread-65 ) Daemon Starting sleep 0.20
[DEBUG] (Thread-66 ) Daemon Starting sleep 0.30
[DEBUG] (MainThread) Thread-2 joining


[<_MainThread(MainThread, started 4670985664)>, <Thread(Thread-2, started daemon 123145391505408)>, <Heartbeat(Thread-3, started daemon 123145396760576)>, <HistorySavingThread(IPythonHistorySavingThread, started 123145403088896)>, <ParentPollerUnix(Thread-1, started daemon 123145408344064)>, <Thread(Thread-64, started daemon 123145413599232)>, <Thread(Thread-65, started daemon 123145418854400)>, <Thread(Thread-66, started daemon 123145424109568)>]


[DEBUG] (Thread-65 ) Daemon Exit
[DEBUG] (Thread-64 ) Daemon Exit
[DEBUG] (Thread-66 ) Daemon Exit


KeyboardInterrupt: 

## 继承thread

In [24]:
class MyThread(threading.Thread):
    def run(self):   # 重写run方法，没有运行target
        logging.debug('this running')
        logging.debug(self._args)
for i in range(5):
    t = MyThread(target=worker, args=('dss','asa'))
    t.start()

[DEBUG] (Thread-87 ) this running
[DEBUG] (Thread-88 ) this running
[DEBUG] (Thread-87 ) ('dss', 'asa')
[DEBUG] (Thread-89 ) this running
[DEBUG] (Thread-90 ) this running
[DEBUG] (Thread-90 ) ('dss', 'asa')
[DEBUG] (Thread-91 ) this running
[DEBUG] (Thread-91 ) ('dss', 'asa')
[DEBUG] (Thread-88 ) ('dss', 'asa')
[DEBUG] (Thread-89 ) ('dss', 'asa')


Timer 提供了一个继承 Thread 的例子，也包含在 threading 模块中。Timer 在延迟一段时间后启动工作，他可以在延迟的这段时间内任何时间点取消。

In [27]:
def delayed():
    logging.debug('worker running')
    
t1 = threading.Timer(0.3, delayed)
t2 = threading.Timer(0.3, delayed)
t1.setName('t1')
t2.setName('t2')
logging.debug('starting timers')
t1.start()
t2.start()

logging.debug('waiting before canceling %s', t2.getName())
time.sleep(0.2)
logging.debug('canceling %s', t2.getName())
t2.cancel()  # 在delay参数内可以取消执行
logging.debug('done')

[DEBUG] (MainThread) starting timers
[DEBUG] (MainThread) waiting before canceling t2
[DEBUG] (MainThread) canceling t2
[DEBUG] (MainThread) done
[DEBUG] (t1        ) worker running


## 线程同步

In [8]:
def wait_for_event(e):
    logging.debug('wait_for_event starting')
    event_is_set = e.wait()  
    logging.debug('event set: %s', event_is_set)
    
def wait_for_event_timeout(e, t):
    while not e.is_set():
        logging.debug('wait_for_event_timeout starting')
        event_is_set = e.wait(t)
        logging.debug('event set: %s', event_is_set)
        if event_is_set:
            logging.debug('processing event')
        else:
            logging.debug('doing other work')

In [9]:
e = threading.Event()
t1 = threading.Thread(name='block', target=wait_for_event, args=(e,))
t2 = threading.Thread(name='nonblock', target=wait_for_event_timeout, args=(e,2))
t1.start()
t2.start()
logging.debug('Waiting before calling event.set()')
time.sleep(0.3)
e.set()
logging.debug('Event is set')

[DEBUG] (block     ) wait_for_event starting
[DEBUG] (nonblock  ) wait_for_event_timeout starting
[DEBUG] (MainThread) Waiting before calling event.set()
[DEBUG] (MainThread) Event is set
[DEBUG] (block     ) event set: True
[DEBUG] (nonblock  ) event set: True
[DEBUG] (nonblock  ) processing event


wait() 方法可以接收一个参数，表示事件等待的超时时间

In [14]:
import random
class Counter:
    def __init__(self, start=0):
        self.lock = threading.Lock()
        self.value = start
    def increment(self):
        logging.debug('Waiting for lock')
        self.lock.acquire()
        try:
            logging.debug('Lock acquired')
            self.value = self.value + 1    # 获取锁后对值操作，保证多个线程同时修改其内部状态不会出问题
        finally:
            self.lock.release()
            
def worker(c):
    for i in range(2):
        pause = random.random()
        logging.debug('Sleeping %0.02f', pause)
        time.sleep(pause)
        c.increment()
    logging.debug('Done')
    
counter = Counter()
for i in range(2):
    t = threading.Thread(target=worker, args=(counter, ), name=str(i))
    t.start()
    
logging.debug('Waiting for work threads')
for t in threading.enumerate():
    if t.getName() == str(0) or t.getName() == str(1):
        t.join()
logging.debug('Count value %d', counter.value)

[DEBUG] (0         ) Sleeping 0.72
[DEBUG] (1         ) Sleeping 0.05
[DEBUG] (MainThread) Waiting for work threads
[DEBUG] (1         ) Waiting for lock
[DEBUG] (1         ) Lock acquired
[DEBUG] (1         ) Sleeping 0.38
[DEBUG] (1         ) Waiting for lock
[DEBUG] (1         ) Lock acquired
[DEBUG] (1         ) Done
[DEBUG] (0         ) Waiting for lock
[DEBUG] (0         ) Lock acquired
[DEBUG] (0         ) Sleeping 0.06
[DEBUG] (0         ) Waiting for lock
[DEBUG] (0         ) Lock acquired
[DEBUG] (0         ) Done
[DEBUG] (MainThread) Count value 4


In [16]:
def lock_holder(lock):
    logging.debug('Starting')
    while True:
        lock.acquire()
        try:
            logging.debug('Holding')
            time.sleep(1)
        finally:
            logging.debug('Not holding')
            lock.release()
        time.sleep(0.5)

def worker(lock):
    logging.debug('work starting')
    num_tries = 0
    num_acquires = 0
    while num_acquires < 3:
        time.sleep(0.5)
        logging.debug('Trying lock acquire')
        have_it = lock.acquire(0) # 从当前线程中得知锁是否被其他线程占用可以向 acquire() 传递 False 来立即得知。
        try:
            num_tries += 1
            if have_it:
                logging.debug('Iteration %d: Acquired', num_tries)
                num_acquires += 1
            else:
                logging.debug('Iteration %d: Not Acquired', num_tries)
        finally:
            if have_it:
                lock.release()
    logging.debug('Done after %d acquires', num_tries)
    
lock = threading.Lock()
t1 = threading.Thread(target=lock_holder, name='lock_holder', args=(lock,), daemon=True)
t2 = threading.Thread(target=worker, name='worker', args=(lock,))          
t1.start()
t2.start()

[DEBUG] (lock_holder) Starting
[DEBUG] (worker    ) work starting
[DEBUG] (lock_holder) Holding
[DEBUG] (lock_holder) Not holding
[DEBUG] (worker    ) Trying lock acquire
[DEBUG] (worker    ) Iteration 1: Not Acquired
[DEBUG] (lock_holder) Holding
[DEBUG] (lock_holder) Not holding
[DEBUG] (worker    ) Trying lock acquire
[DEBUG] (worker    ) Iteration 2: Acquired
[DEBUG] (lock_holder) Holding
[DEBUG] (worker    ) Trying lock acquire
[DEBUG] (worker    ) Iteration 3: Not Acquired
[DEBUG] (lock_holder) Not holding
[DEBUG] (worker    ) Trying lock acquire
[DEBUG] (worker    ) Iteration 4: Not Acquired
[DEBUG] (lock_holder) Holding
[DEBUG] (lock_holder) Not holding
[DEBUG] (worker    ) Trying lock acquire
[DEBUG] (worker    ) Iteration 5: Acquired
[DEBUG] (lock_holder) Holding
[DEBUG] (worker    ) Trying lock acquire
[DEBUG] (worker    ) Iteration 6: Not Acquired
[DEBUG] (lock_holder) Not holding
[DEBUG] (worker    ) Trying lock acquire
[DEBUG] (worker    ) Iteration 7: Not Acquired
[DEBUG

为了获取三次锁而尝试了8次。

In [1]:
import threading
lock = threading.Lock()
print('first try,', lock.acquire())
print('second try,', lock.acquire(0))

rlock = threading.RLock()
print('first try,', rlock.acquire())
print('second try,', rlock.acquire(0))
print('third try,', rlock.acquire(0))
# acquire() 两次，release() 也要两次

first try, True
second try, False
first try, True
second try, True
third try, True


In [2]:
import logging
logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)
# worker_with() 和 worker_no_with() 所实现的功能一模一样的。
def worker_with(lock):
    with lock:
        logging.debug('lock acquired via with')
def worker_no_with(lock):
    lock.acquire()
    try:
        logging.debug('lock acquired via try')
    finally:
        lock.release()
lock = threading.Lock()
w = threading.Thread(target=worker_with, args=(lock,))
nw = threading.Thread(target=worker_no_with, args=(lock,))

w.start()
nw.start()

(Thread-4  ) lock acquired via with
(Thread-5  ) lock acquired via try


In [9]:
import time
def consumer(cond):
    logging.debug('Starting consumer')
    with cond:
        cond.wait()
        logging.debug('Resource is available to consumer')
def producer(cond):
    logging.debug('Starting producer')
    with cond:
        
        cond.notifyAll()
        logging.debug('making resource available1')
        
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s (%(threadName)-2s) %(message)s',
)
condition = threading.Condition()
c1 = threading.Thread(target=consumer, args=(condition, ), name='c1')
c2 = threading.Thread(target=consumer, args=(condition, ), name='c2')
p = threading.Thread(target=producer, args=(condition, ), name='p')
c1.start()
time.sleep(2)
c2.start()
time.sleep(2)
p.start()

(c1        ) Starting consumer
(c2        ) Starting consumer
(p         ) Starting producer
(p         ) making resource available1
(c1        ) Resource is available to consumer
(c2        ) Resource is available to consumer


除了使用 Events，另一种同步线程的方法是使用 Condition 对象。 Condition 使用了 Lock，所以它会绑定共享的资源，也就会让多个线程等待资源更新完成。

屏障」（Barrier）是另一种线程同步的机制。每个 Barrier 会建立起一个控制点，所有处在其中的线程都会被阻塞，直到所有的线程都到达这个控制点。它会让所有的线程单独启动，然后在它们全都准备好执行下一步前先阻塞住。

In [12]:
def worker(barrier):
    print(threading.current_thread().name, f"wait for barrier with {barrier.n_waiting} others")
    worker_id = barrier.wait()
    print(f"{threading.current_thread().name} after barrier  {worker_id}")

NUM_T = 3
barrier = threading.Barrier(NUM_T)
threads = [threading.Thread(name=f'worker-{i}', target=worker, args=(barrier,)) for i in range(NUM_T)]
for t in threads:
    print(t.name, 'starting')
    t.start()
    time.sleep(0.1)

for t in threads:
    t.join()

worker-0 starting
worker-0 wait for barrier with 0 others
worker-1 starting
worker-1 wait for barrier with 1 others
worker-2 starting
worker-2 wait for barrier with 2 others
worker-2 after barrier  2
worker-0 after barrier  0
worker-1 after barrier  1


In [14]:
def worker(barrier):
    print(threading.current_thread().name, f"wait for barrier with {barrier.n_waiting} others")
    try:
        worker_id = barrier.wait()
    except threading.BrokenBarrierError:
        print(f'{threading.current_thread().name} aborting')
    else:
        print(f"{threading.current_thread().name} after barrier  {worker_id}")

NUM_T = 3
barrier = threading.Barrier(NUM_T + 1)
threads = [threading.Thread(name=f'worker-{i}', target=worker, args=(barrier,)) for i in range(NUM_T)]
for t in threads:
    print(t.name, 'starting')
    t.start()
    time.sleep(0.1)
barrier.abort()
for t in threads:
    t.join()

worker-0 starting
worker-0 wait for barrier with 0 others
worker-1 starting
worker-1 wait for barrier with 1 others
worker-2 starting
worker-2 wait for barrier with 2 others
worker-0 abortingworker-1 aborting
worker-2 aborting



Barrier 的 abort() 方法会导致所有等待中的线程接收到一个 BrokenBarrierError。 我们可以使用此方法来告知那些被阻塞住的线程该结束了。这次我们将 Barrier 设置成比实际开始的线程多一个，这样所有的线程就会被阻塞住，我们调用 abort() 就可以引起 BrokenBarrierError 了。

In [16]:
class ActivePool:
    def __init__(self):
        self.active = []
        self.lock = threading.Lock()
    def make_active(self, name):
        with self.lock:
            self.active.append(name)
            logging.debug(f"Running {self.active}")
    def make_inactive(self, name):
        with self.lock:
            self.active.remove(name)
            logging.debug(f"Running {self.active}")
            
def worker(s, pool):
    logging.debug('Wait for join the pool')
    with s:
        name = threading.current_thread().getName()
        pool.make_active(name)
        time.sleep(1)
        pool.make_inactive(name)
        
pool = ActivePool()
s = threading.Semaphore(2)

for i in range(4):
    t = threading.Thread(target=worker, name=str(i), args=(s, pool))
    t.start()
        

(0         ) Wait for join the pool
(1         ) Wait for join the pool
(2         ) Wait for join the pool
(0         ) Running ['0']
(3         ) Wait for join the pool
(1         ) Running ['0', '1']
(0         ) Running ['1']
(2         ) Running ['1', '2']
(1         ) Running ['2']
(3         ) Running ['2', '3']
(2         ) Running ['3']
(3         ) Running []


有时我们需要允许多个工作函数在同一时间访问同一个资源，但我们也要限制可访问的总数。  
ActivePool 类只是用来追踪给定时刻下哪些线程在工作的。如果是实际情况中，资源池一般还要分配连接或者其他值给新的活动线程，并且当线程结束后回收这些值。

## thread local

In [18]:
import random
def show_value(data):
    try:
        val=data.value
    except AttributeError:
        logging.debug('Not value yet')
    else:
        logging.debug('value=%s', val)
        
def worker(data):
    show_value(data)
    data.value = random.randint(1,100)
    show_value(data)
    
local_data = threading.local()
show_value(local_data)
local_data.value = 1000
show_value(local_data)

for i in range(2):
    t = threading.Thread(target=worker, args=(local_data,))
    t.start()

(MainThread) Not value yet
(MainThread) value=1000
(Thread-8  ) Not value yet
(Thread-9  ) Not value yet
(Thread-8  ) value=10
(Thread-9  ) value=25


local() 类可以在每个线程中创建一个用于隐藏值的对象容器。 local_data.value 在当前的线程设置任何值前，对于当前线程来说它都什么都没有。

In [19]:
class MyLocal(threading.local):
    def __init__(self, value):
        super().__init__()
        logging.debug('Initializing %r', self)
        self.value = value
        
    
local_data = MyLocal(1000)
show_value(local_data)

for i in range(2):
    t = threading.Thread(target=worker, args=(local_data,))
    t.start()

(MainThread) Initializing <__main__.MyLocal object at 0x107e93d68>
(MainThread) value=1000
(Thread-10 ) Initializing <__main__.MyLocal object at 0x107e93d68>
(Thread-11 ) Initializing <__main__.MyLocal object at 0x107e93d68>
(Thread-10 ) value=1000
(Thread-11 ) value=1000
(Thread-10 ) value=44
(Thread-11 ) value=12
